In [26]:
import os
import re
import numpy as np
import pandas as pd
from datetime import date, datetime
from calendar import monthrange
from IPython.display import display,HTML
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
from DataProcessing import import_data,property_input,format_reservation
data = import_data()
property=property_input()

In [27]:
osbr = data[data["Listing"].str.contains("Cottage")]
mask = (osbr["checkin_date"]>=pd.to_datetime('2025-11-01')) & (osbr["checkin_date"]<=pd.to_datetime('2025-11-30')) |\
(osbr["checkout_date"]>=pd.to_datetime('2025-11-01')) & (osbr["checkout_date"]<=pd.to_datetime('2025-11-30'))
osbr = osbr.loc[mask]
checkins = osbr.assign(Date = osbr["checkin_date"].astype(str).str[:10]).groupby("Date", as_index=False).agg(checkin=("checkin_date",'count'))
checkouts = osbr.assign(Date = osbr["checkout_date"].astype(str).str[:10]).groupby("Date", as_index=False).agg(checkout=("checkout_date",'count'))
counts = checkins.merge(checkouts,on="Date",how="outer")
df = pd.DataFrame({
    "Date": pd.date_range("2025-11-01", "2025-11-30")
})
counts["Date"] = pd.to_datetime(counts["Date"])
counts = df.merge(counts,on="Date",how="left")
counts["Weekday"] = counts["Date"].dt.day_name()
counts["Date"] = counts["Date"].astype(str).str[:10]
counts = counts[["Date", "Weekday", "checkin", "checkout"]]

In [28]:
osbr_sum = (osbr[["Listing","checkout_date"]].merge(property[["Listing","BEDROOMS"]],on="Listing",how="left")
            .pivot_table(index="checkout_date", columns="BEDROOMS",aggfunc="size",fill_value=0).reset_index()
            .assign(Date=lambda x: x["checkout_date"].astype(str).str[:10]))

counts = counts.merge(osbr_sum, on="Date", how="left").drop(columns=["checkout_date"])
counts.to_excel("/Users/ylin/My Drive/Cohost/Data and Reporting/05-Cleaning/Brittany_booking_counts.xlsx", index=False,na_rep="")

In [34]:
ratings_path = "/Users/ylin/My Drive/Cohost/** Properties ** -- Valta/0_Cohosting/1-Reviews/Guesty reviews from Tech team/20251201 guesty_reviews.xlsx"
ratings = pd.read_excel(ratings_path)
ratings["createdAt"] = pd.to_datetime(ratings["createdAt"], errors="coerce")
idx = (
    ratings["nickname"].str.contains("Cottage", na=False)
    & (ratings["createdAt"] > pd.to_datetime("2025-10-31"))
    & (ratings["createdAt"] < pd.to_datetime("2025-12-01")) 
    & (ratings["Overall"].isin([5,10]))
)
ratings.loc[idx]

,Unnamed: 0,reservationId,channelId,createdAt,nickname,Property,Guest name,Check in,Check out,Accuracy,...,Location,Value,Overall,Public Review,Reservation,Booking.com Rating,Booking.com Positive content,Booking.com Negative content,Cohost review,Cohost
7,7,691563075fbb68004458b541,airbnb2,2025-11-25,Cottage 2,NaN,Sean,2025-11-14,2025-11-15,5.0,...,5.0,5.0,5.0,Cabin was just like pictures. Very well kept. Enjoyed the rec room which was really nice space. Loved the Grayland area and was a quiet neighborhood. I look forward to another visit there next year.,HMNCCWNWXQ,NaN,NaN,NaN,0,NaN
10,10,68edc8ec2bcd3a001a0c8af8,bookingCom,2025-11-23,Cottage 2,NaN,Lisa,2025-11-19,2025-11-21,10.0,...,10.0,10.0,10.0,NaN,BC-kg2ngMLPr,10.0,NaN,NaN,0,NaN
14,14,690c5f5b4c2c810011a96d1d,airbnb2,2025-11-21,Cottage 2,NaN,Brayden,2025-11-15,2025-11-17,4.0,...,5.0,4.0,5.0,"It was definitely worth the cost and the views are amazing, beach access close by as well so that was a plus!! Overall we really enjoyed our stay!",HMTCE9DDPZ,NaN,NaN,NaN,0,NaN
18,18,69094bd0f27bbb00131bdef3,airbnb2,2025-11-18,Cottage 6,NaN,Ashley,2025-11-08,2025-11-09,5.0,...,5.0,5.0,5.0,Perfect stay for our weekend crabbing trip! Cozy and clean!,HMSAT2A3DH,NaN,NaN,NaN,0,NaN
40,40,68d7697ac3466000126697e4,airbnb2,2025-11-12,Cottage 1,NaN,Becky,2025-11-08,2025-11-11,5.0,...,5.0,5.0,5.0,What a wonderful get away! The cottage was perfectly cozy and I love having the beach so close!,HMZFATXTK3,NaN,NaN,NaN,0,NaN
49,49,6907a8c397f5d2000f218008,airbnb2,2025-11-11,Cottage 4,NaN,Lori,2025-11-07,2025-11-08,5.0,...,5.0,5.0,5.0,"Comfortable beds, nice towels, cute place would stay there again.",HMWT8WZX55,NaN,NaN,NaN,0,NaN
51,51,690e6ad669fb630013910e70,airbnb2,2025-11-11,Cottage 11 (tiny),NaN,Chase,2025-11-08,2025-11-10,5.0,...,5.0,5.0,5.0,Beautiful little unit for the weekend! loved the soaker tup! Thanks for another great stay!,HMBPAF3TYH,NaN,NaN,NaN,0,NaN
55,55,690a9c4c6ba07f0012486e1c,homeaway2-Vrbo,2025-11-09,Cottage 12,NaN,James,2025-11-07,2025-11-09,5.0,...,5.0,NaN,5.0,Host was very helpful. We loved the close access to the beach. Pet friendly is always helpful! We enjoyed our stay!,HA-IHBK1XY,NaN,NaN,NaN,0,NaN
59,59,68c9a382ea829a00322ff3e8,airbnb2,2025-11-08,Cottage 2,NaN,Jeff,2025-11-04,2025-11-07,5.0,...,5.0,5.0,5.0,With king tides and great razor clamming I couldn't ask for more.,HMYYEQK9J2,NaN,NaN,NaN,0,NaN
61,61,68ee6bfeec7cd2001241df75,airbnb2,2025-11-07,Cottage 6,NaN,Tim,2025-11-04,2025-11-07,5.0,...,5.0,5.0,5.0,"It is alway great to stay at Ocean Spray. We have stayed in many different cabins, but do have our favorite. They are all good.\nThey are always one step ahead of us on all communications\nAgain, an awesome stay\nThank You",HMAZ33APJH,NaN,NaN,NaN,0,NaN
